In [2]:
import pandas as pd
import re
from flatten_json import flatten
import json
from load_data_to_clean import load_data_to_clean

In [3]:
raw_data, dataset_names = load_data_to_clean()

Loaded met with shape: (476318, 54)
Loaded reina_sofia with shape: (10018, 16)
Loaded tate with shape: (69201, 31)
Loaded pompidou with shape: (115217, 23)
Loaded moma with shape: (157457, 30)
Loaded whitney with shape: (26495, 20)
Loaded national_gallery with shape: (141430, 55)
Loaded flattened data with shape: (58264, 499)
Loaded flattened data with shape: (175000, 2852)
Loaded flattened data with shape: (58264, 499)
Loaded queensland with shape: (20250, 17)


create functions for cleaning fields.
artist name, artwork title, artwork medium, and acquisition source, aquisition year, creation year, artist nationality, artist gender

In [4]:
def create_artist_name(df):
    # List of possible column names for the artist name
    possible_columns = ['artist', 'author', 'artist_name', 'Artist Display Name', 'artist name', 'Artist Display Name', 'Artist', 'artists', 
                        'forwarddisplayname', 'production_0_creator']
    
    # Loop over the possible column names and use the first one that exists
    for col in possible_columns:
        if col in df.columns:
            df['Artist'] = df[col]
            break  # Exit loop once we find the first match
    
    return df

In [5]:
def create_artwork_title(df):
    # List of possible column names for the artist name
    possible_columns = ['object_title', 'artwork_name', 'Title', 'url', 'name', 'title', 'title_fi', 'titles_0_title']
    
    # Loop over the possible column names and use the first one that exists
    for col in possible_columns:
        if col in df.columns:
            df['Title'] = df[col]
            break  # Exit loop once we find the first match
    
    return df

In [7]:
from extract_first_number import extract_first_number

    
def clean_acquisition_year(df):
   # List of possible column names for the artist name
    possible_columns = ['acquisition_date', 'credit_line', 'AccessionYear', 'artwork_acquisition', 'year_adquisition', 'acquisitionYear',
                        'DateAcquired', 'accession_number', 'accessionnum', 'inventoryNumber', 'acquisition_date_precision', 'AcquiredDate' ]
    df['Year_acquisition'] = pd.NA
  
    # Loop over the possible column names and use the first one that exists
    for col in possible_columns:
        if col in df.columns:
            df['Year_acquisition'] = df[col].apply(extract_first_number)
            break  # Exit loop once we find the first match
    
    return df 

In [23]:
def create_artwork_medium(df):
    # List of possible column names for the artist name
    possible_columns = ['Classification', 'classification', 'classifications_0_en','object_type', 'type_artwork', 'artwork_medium', 'Medium', 'medium',
                        'object_names_1_name', 'PhysicalCategory']
    # Loop over the possible column names and use the first one that exists
    for col in possible_columns:
        if col in df.columns:
            df['Medium_raw'] = df[col]
            break  # Exit loop once we find the first match
    return df

In [ ]:
#I need to add more tags based on dataset values
from medium_tags import load_medium_tags #returns medium_tags, medium_name
medium_tags, medium_name = load_medium_tags()
def classify_medium(df):
    df['Medium_classified'] = pd.NA
    
     # Iterate over each row in the DataFrame
    for idx, row in df.iterrows():
        medium = row['Medium_raw'] 
        
        # Skip if the 'Medium' value is not a string (e.g., NaN or other types)
        if not isinstance(medium, str):
            continue
        
        # Check each tag group (e.g., ['paper', 'watercolor'], ['oil'])
        for i, tag_group in enumerate(medium_tags):
            # Check if any keyword from the tag group exists in the 'Medium' column
            if any(tag.lower() in medium.lower() for tag in tag_group):
                # Assign the corresponding medium name (from medium_name list)
                df.at[idx, 'Medium_classified'] = medium_name[i]
                break  # Stop once we find a match
    
    return df

In [88]:
test = transformed_data[1][['nationality_raw', 'Country_calculated']].drop_duplicates(subset=['nationality_raw'])



In [85]:
transformed_data[10]['nationality_raw'].unique()

array(['queensland australia ', 'new south wales australia ',
       'lithuania/australia vic ', ..., 'togo-brisby, jasmine - creator',
       'savage, paula - creator', 'ryui, koji - creator'], dtype=object)

In [ ]:
#i need to update the dictionary so all nationalities will be translated into countries
from load_nationality_tags import load_nationality_tags
nationality_tags, country_name = load_nationality_tags()
def create_artist_nationality(df):
    possible_columns = ['Country','Artist Nationality', 'nationality_artist', 'artist_nationality', 'Nationality', 'nationality', 
                        'production_0_creator_nationality']
    df['Country_calculated'] = pd.NA
    df['nationality_raw'] = pd.NA

    # Check each possible column for nationality information
    for col in possible_columns:
        if col in df.columns:
            df['nationality_raw'] = df[col].str.replace('(', '').str.replace(')', '').str.lower()  # Copy the nationality info into 'nationality_raw'
            break  # Exit loop once we find the first match 

    for idx, row in df.iterrows():
        nationality = row['nationality_raw'] 
        
        # Skip if the 'nationality' value is not a string (e.g., NaN or other types)
        if not isinstance(nationality, str):
            continue
        
        # Check each tag group (e.g., ['paper', 'watercolor'], ['oil'])
        for i, tag_group in enumerate(nationality_tags):
            # Check if any keyword from the tag group exists in the 'nationality' column
            if any(tag.lower() in nationality.lower() for tag in tag_group):
                # Assign the corresponding nationality name (from nationality_name list)
                df.at[idx, 'Country_calculated'] = country_name[i]
                break  # Stop once we find a match

    return df


use functions for raw datasets and dreate a new list with clean datasets

In [ ]:

def create_artist_gender(df):
    possible_columns = ['Artist Gender', 'gender', 'artist_gender', 'Gender', 'production_0_creator_gender']
    # Loop over the possible column names and use the first one that exists
    gender_dict = {'Female':"female",
                   'male':'male'}
    for col in possible_columns:
        if col in df.columns:
            df['Gender_calculated'] = df[col].replace(gender_dict)
            break  # Exit loop once we find the first match
    
    return df

In [ ]:
def create_acquisition_method(df):
    possible_columns = ['Credit Line', 'creditLine', 'acquisition_type', 'CreditLine', 'credit_line', 'creditline', 'CreditLine']

In [ ]:
def artwork_creation_date(df):
    possible_columns = ['Object Date', 'year_production', 'year', 'object_date', 'Date', 'display_date', 'endyear_x', 'yearFrom', 
                        'production_date_0_end', 'DateCreated']
    
    df['Date_creation_year'] = pd.NA
  
    # Loop over the possible column names and use the first one that exists
    for col in possible_columns:
        if col in df.columns:
            df['Date_creation_year'] = df[col].apply(extract_first_number)
            break  # Exit loop once we find the first match
    
    return df 

In [90]:
def artist_birth_year(df):
    possible_columns = ['Artist Begin Date', 'author_born_year', 'yearOfBirth', 'artist_birth', 'BeginDate', 'birth_date', 
                        'beginyear', 'people_0_birthDate', 'production_0_creator_date_of_birth']
    df['Artist_birth_year'] = pd.NA
  
    # Loop over the possible column names and use the first one that exists
    for col in possible_columns:
        if col in df.columns:
            df['Artist_birth_year'] = df[col].apply(extract_numbers)
            break  # Exit loop once we find the first match
    
    return df 
    

In [92]:
def artist_death_year(df):
    possible_columns = ['Artist End Date', 'author_death_year', 'yearOfDeath', 'artist_death', 'EndDate', 'death_date', 'endyear', 
                        'people_0_deathDate', 'production_0_creator_date_of_death']
    df['Artist_death_year'] = pd.NA
  
    # Loop over the possible column names and use the first one that exists
    for col in possible_columns:
        if col in df.columns:
            df['Artist_death_year'] = df[col].apply(extract_numbers)
            break  # Exit loop once we find the first match
    
    return df

In [86]:
transformed_data = []
for df in raw_data:
    #create_artist_name(df)
    #create_artwork_title(df)
    #create_artwork_medium(df)
    #classify_medium(df)
    create_artist_nationality(df)
    #artwork_creation_date(df)
    #clean_acquisition_year(df)
    #artist_birth_year(df)
    #artist_death_year(df)
    #create_artist_gender(df)
    transformed_data.append(df)


In [83]:
transformed_data[0]['Gender_calculated'].unique()

array([nan, '|', '|Female', '||', '||||', 'Female|', 'Female', '|||',
       'Female|Female', '||Female', '|Female|Female', 'Female||',
       'Female|Female|Female|Female|Female||Female|Female||||||Female|Female|Female|Female|||Female|||Female|Female|',
       '|Female|', '|||Female', 'Female||Female', 'Female|Female|Female|',
       '||||||||', '|||||||||||||||||||||||||||', 'Female|Female|',
       'Female|Female||', 'Female|||', '|Female||', '|Female||Female',
       '||Female|', 'Female|Female|Female', '||||||', '|Female|||Female',
       '|||||', '||||Female|', '||||||||||', '||||||||||||||||',
       '|||||||', '|||||||||||||', '|||||||||', '||||||||||||||||||||',
       'Female|||||||||||||||||||||||||||||Female||Female||||',
       'Female||||Female|||Female|||||||||||||||||Female|',
       '|Female||||||||||||||Female', '||||||Female||||||',
       'Female||||Female|||||Female|||',
       '||||Female||||||Female|||||Female||||Female', '||||Female',
       '|||||||||||||||||||

save clean datasets in separate csv files

In [ ]:
#i need to add more colums to the final dataset
clean_data = []
for df in transformed_data:
    df = df[[ 'Artist', 'Title','Medium', 'Year_acquisition', 'Medium_classified']]
    clean_data.append(df)

In [ ]:

# Save each dataset using the corresponding name
for dataset, name in zip(clean_data, dataset_names):
    # Generate a unique filename using the dataset name
    file_name = f"{name}.csv"
    
    # Save the DataFrame to a CSV file
    dataset.to_csv(file_name, index=False)
    print(f"Saved {file_name}")

Saved met_clean.csv
Saved reina_sofia_clean.csv
Saved tate_clean.csv
